This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [44]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [45]:
# First, import the relevant modules
import requests

In [46]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2019-07-10&api_key=' + API_KEY)

In [47]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset_data': {'limit': None,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2019-07-10',
  'end_date': '2019-07-10',
  'frequency': 'daily',
  'data': [['2019-07-10',
    86.05,
    86.55,
    84.15,
    85.75,
    None,
    116226.0,
    9947223.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [48]:
# This code performs the get request from Quandl for the specified data we need
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY)

In [49]:
# This code uses list comprehension to create a list of column names from the JSON data
cols_list = [col for col in r.json()['dataset_data']['column_names']]

In [50]:
# This code uses list comprehension to create a list of data values from the JSON data
vals_list = [sublist for sublist in r.json()['dataset_data']['data']]

In [51]:
# This code uses dictionary comprehension to create a nested dictionary from the cols_list and vals_list
afx_dict = {val[0]:{cols_list[i]:val[i] for i in range(1,11)} for val in vals_list}

In [52]:
# This code uses list comprehension to go through the nested dictionary creating a list of opening 
# prices throughout 2017. I have filtered this list to remove None type values for the purpose of finding 
# the max and min.
open_list = [afx_dict[k]['Open'] for k in afx_dict.keys()]
open_list = list(filter(None, open_list))

In [53]:
# It appears the maximum opening price in 2017 was 53.11
max(open_list)

53.11

In [54]:
# The minimum opening opening price in 2017 was 34.0
min(open_list)

34.0

In [55]:
# This code creates a list of intraday prince changes
intraday_change_list = [afx_dict[k]['High'] - afx_dict[k]['Low'] for k in afx_dict.keys()]

In [56]:
# The maximum price change within one day in 2017 was 2.81
max(intraday_change_list)

2.8100000000000023

In [57]:
# This code creates a list of closing prices, and then proceeds to create a new list which is the difference
# between subsequent elements of the closing price list. This yields a list of changes in closing price from
# one trading day to the next.
daily_close_list = [afx_dict[k]['Close'] for k in afx_dict.keys()]
daily_change_list = [daily_close_list[i+1]-daily_close_list[i] for i in range(len(daily_close_list)-1)]

In [58]:
# It appears the maximum increase from one day to the next is 2.56.
max(daily_change_list)

2.559999999999995

In [59]:
# The maximum decrease is -1.72, which is smaller in absolute value than 2.56. Therefore, the largest
# change in closing price from one trading day to the next was an increase of 2.56.
min(daily_change_list)

-1.7199999999999989

In [60]:
# Once again we create a list from the nested dictionary; this time of daily trading volume.
volume_list = [afx_dict[k]['Traded Volume'] for k in afx_dict.keys()]

In [61]:
# We calculate the average by summing all daily trading volumes and dividing by the number of trading days.
# The average daily trading volume in 2017 was 89124.
sum(volume_list)/len(volume_list)

89124.33725490196

In [62]:
# This code defines a function to calculate the median of a list of numbers. It does so naively by recursively 
# finding the max and min of the list and removing them until there are only one or two elements left. It then
# takes the average of the remaining elements in case there are two remaining elements.
def median(input_list):
    while len(input_list) > 2:
        input_list.remove(max(input_list))
        input_list.remove(min(input_list))
    return sum(input_list)/len(input_list)

In [63]:
# It appears the median daily trading volume through 2017 was 76286.
median(volume_list)

76286.0